In [1]:
import nltk
nltk.download('punkt')

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model

[nltk_data] Downloading package punkt to C:\Users\ADITYA
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df=pd.read_csv(r"C:\Users\ADITYA SINGH\Downloads\NLP_train.csv")
print(df.head())

                 id                                       comment_text  toxic
0  e617e2489abe9bca  "\r\n\r\n A barnstar for you! \r\n\r\n  The De...      0
1  9250cf637294e09d  "\r\n\r\nThis seems unbalanced.  whatever I ha...      0
2  ce1aa4592d5240ca  Marya Dzmitruk was born in Minsk, Belarus in M...      0
3  48105766ff7f075b      "\r\n\r\nTalkback\r\n\r\n Dear Celestia...  "      0
4  0543d4f82e5470b6  New Categories \r\n\r\nI honestly think that w...      0


In [3]:
df.drop(columns = ['id'], inplace=True)
df.head()

,comment_text,toxic
0,"""\r\n\r\n A barnstar for you! \r\n\r\n The De...",0
1,"""\r\n\r\nThis seems unbalanced. whatever I ha...",0
2,"Marya Dzmitruk was born in Minsk, Belarus in M...",0
3,"""\r\n\r\nTalkback\r\n\r\n Dear Celestia... """,0
4,New Categories \r\n\r\nI honestly think that w...,0


In [4]:
df.comment_text[2]

'Marya Dzmitruk was born in Minsk, Belarus in March 19, 1992. Her mother, Olga Nikolaevna Moroz was born in Baranovichi, Belarus and her father was born in Brest, Belarus. She is second child in the family. Her parents divorced in 1998 and soon after her father remarried and had two more children. \r\nMarya, at the age of 4 began doing gymnastics, but quit two years later because she was denied a medal in a competition, where her age was incorrectly marked. When she turned 6 years old, she got admitted to Music School #4 in Minsk, class of violin, and to Public School #66 with piano classes as a main course. At the age of 11, Marya starred in Belarusfilm movie called “Dunechka”. Soon after she started to play in theatre and was featured in television shows. By 2005 her mother decided to move to United States. In September of 2005 Marya went to her first American school, Ingrid B. Lacy Middle School. She graduated in Spring 2006 and traveled back to Belarus for 2 months. In August 2006 

In [5]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
    

[nltk_data] Downloading package stopwords to C:\Users\ADITYA
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
            
    return result

In [7]:
df['clean'] = df['comment_text'].apply(preprocess)

In [8]:
df.clean[0]

['barnstar',
 'defender',
 'wiki',
 'barnstar',
 'like',
 'edit',
 'kayastha',
 'page',
 'lets',
 'form',
 'solidarity',
 'group',
 'malign',
 'article',
 'subject',
 'matter',
 'propose',
 'folloing',
 'group',
 'united',
 'intellectuals',
 'kayastha',
 'ethinicty',
 'racist',
 'castist',
 'abuse',
 'uifkearca']

In [9]:
print(type(df.clean[0]),df.comment_text[0])

<class 'list'> "

 A barnstar for you! 

  The Defender of the Wiki Barnstar I like your edit on the Kayastha page. Lets form a solidarity group against those who malign the article and its subject matter. I propose the folloing name for the group.

United intellectuals' front of Kayastha ethinicty against racist or castist abuse (UIFKEARCA)   "


In [10]:
list_of_words = []
for i in df.clean:
    for j in i:
        list_of_words.append(j)
total_words = (list(set(list_of_words)))
print(len(total_words))

20047


In [11]:
print(len(list_of_words))

136400


In [12]:
maxlen=0
for i in df.clean:
    maxlen=max(maxlen,len(i))
print(maxlen)

712


In [13]:
counter={}
c=0
for i in total_words:
    c=0
    for j in df.clean:
        for k in j:
            if i==k:
                c=c+1
    counter[i]=c

In [14]:
print(counter)

{'sink': 2, 'contemplated': 1, 'acceptance': 5, 'designers': 2, 'pido': 1, 'organization': 35, 'ofensive': 1, 'demolish': 1, 'moos': 1, 'disconnected': 1, 'requirements': 11, 'biggie': 1, 'calm': 11, 'decider': 3, 'tabloid': 1, 'payable': 2, 'sympathy': 1, 'populations': 4, 'momusufan': 1, 'markup': 3, 'peacock': 2, 'tits': 3, 'torches': 1, 'messages': 46, 'expect': 32, 'cicero': 1, 'inuit': 1, 'relativement': 1, 'happen': 45, 'sthenelaidas': 3, 'immunity': 1, 'ritual': 1, 'busy': 14, 'redefinition': 1, 'forked': 1, 'angelofsadness': 1, 'irrelelevant': 1, 'pics': 4, 'randomly': 2, 'gadgetbahn': 1, 'deliberately': 8, 'rosja': 1, 'parliaments': 3, 'slashdotted': 1, 'latin': 23, 'samoa': 1, 'thwarted': 1, 'adminship': 7, 'holocaustic': 1, 'invoved': 1, 'islamism': 1, 'evolution': 27, 'pederasty': 1, 'wperdue': 1, 'unrelated': 7, 'narrowly': 1, 'dominates': 1, 'illathu': 1, 'award': 32, 'toss': 2, 'thing': 180, 'moore': 3, 'harris': 1, 'sparse': 2, 'mets': 1, 'deleet': 1, 'legislated': 1, 

In [15]:
df['clean_joined'] = df['clean'].apply(lambda x: " ".join(x))

In [16]:
from nltk import word_tokenize
tokenizer = Tokenizer(num_words = len(total_words))
tokenizer.fit_on_texts(df.clean_joined)

In [17]:
df['tokenized']= tokenizer.texts_to_sequences(df.clean_joined)

In [18]:
co=0
for i in range(len(df.toxic)):
    if df.toxic[i]==1:
        co=co+1
print(co,len(df.toxic))

437 5000


In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.tokenized, df.toxic, test_size = 0.3)

In [20]:
print(df.tokenized[0])

[613, 6604, 34, 613, 6, 9, 6605, 2, 1528, 231, 9653, 189, 9654, 1, 47, 153, 996, 9655, 189, 258, 9656, 6605, 9657, 705, 9658, 517, 9659]


In [21]:
print(df.clean_joined[0])

barnstar defender wiki barnstar like edit kayastha page lets form solidarity group malign article subject matter propose folloing group united intellectuals kayastha ethinicty racist castist abuse uifkearca


In [22]:
df.toxic


0       0
1       0
2       0
3       0
4       0
       ..
4995    0
4996    1
4997    0
4998    0
4999    0
Name: toxic, Length: 5000, dtype: int64

In [23]:
padded_train = pad_sequences(x_train,maxlen = 40, padding = 'post', truncating = 'post')
padded_train= padded_train.reshape(3500,40)
padded_test = pad_sequences(x_test,maxlen = 40, truncating = 'post')
padded_test = padded_test.reshape(1500,40)

In [24]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train1 = sc.fit_transform(padded_train)
x_test1 = sc.transform(padded_test)

In [25]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(x_train1, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [26]:
y_pred = classifier.predict(x_test1)

In [27]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1382,    0],
       [ 118,    0]], dtype=int64)

In [28]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
acc=accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred)
rec=recall_score(y_test,y_pred)
print(acc,f1,rec)

0.9213333333333333 0.0 0.0


# next part as model was only focussing on 0's

In [29]:
cl=SVC( kernel='linear', class_weight={1: 9})
cl.fit(x_train1, y_train)
y_pred=cl.predict(x_test1)
cm1=confusion_matrix(y_test, y_pred)
cm1

array([[968, 414],
       [ 83,  35]], dtype=int64)

In [30]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
acc=accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred)
rec=recall_score(y_test,y_pred)
print(acc,f1,rec)

0.6686666666666666 0.12345679012345678 0.2966101694915254


In [31]:
clf = SVC(kernel='linear', C=1.0)
clf.fit(x_train1, y_train)
y_pred=clf.predict(x_test1)
cm1=confusion_matrix(y_test, y_pred)
cm1

array([[1382,    0],
       [ 118,    0]], dtype=int64)

In [32]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
acc=accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred)
rec=recall_score(y_test,y_pred)
print(acc,f1,rec)

0.9213333333333333 0.0 0.0


In [33]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
clfr = GridSearchCV(SVC(kernel='linear', gamma='auto',class_weight={1: 9}), {
    'C': [1,10,20,30,40,50,60]
}, cv=5, return_train_score=False)
clfr.fit(x_train1, y_train)
data= pd.DataFrame(clfr.cv_results_)
data

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.976365,0.116350,0.053502,0.002878,1,{'C': 1},0.487143,0.527143,0.408571,0.508571,0.565714,0.499429,0.052252,7
1,4.477048,0.508324,0.055783,0.004619,10,{'C': 10},0.485714,0.535714,0.408571,0.508571,0.567143,0.501143,0.053691,1
2,9.055334,2.385289,0.056466,0.003278,20,{'C': 20},0.485714,0.535714,0.408571,0.508571,0.564286,0.500571,0.052997,3
3,14.592474,3.691591,0.069787,0.025206,30,{'C': 30},0.485714,0.535714,0.408571,0.508571,0.565714,0.500857,0.053342,2
4,18.490927,3.859439,0.083060,0.034617,40,{'C': 40},0.485714,0.532857,0.408571,0.510000,0.565714,0.500571,0.053028,3
5,25.243290,7.130110,0.083024,0.034552,50,{'C': 50},0.485714,0.534286,0.408571,0.508571,0.564286,0.500286,0.052810,5
6,29.968091,6.150423,0.056452,0.002522,60,{'C': 60},0.485714,0.532857,0.408571,0.508571,0.565714,0.500286,0.052980,6


In [34]:
print(data[['param_C','mean_test_score']])
print(clfr.best_params_, clfr.best_score_)

  param_C  mean_test_score
0       1         0.499429
1      10         0.501143
2      20         0.500571
3      30         0.500857
4      40         0.500571
5      50         0.500286
6      60         0.500286
{'C': 10} 0.5011428571428571


In [35]:
cl=SVC( kernel='linear', class_weight={1: 9}, C=30)
cl.fit(x_train1, y_train)
y_pred=cl.predict(x_test1)
cm1=confusion_matrix(y_test, y_pred)
cm1

array([[971, 411],
       [ 83,  35]], dtype=int64)

In [36]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
acc=accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred)
rec=recall_score(y_test,y_pred)
print(acc,f1,rec)

0.6706666666666666 0.12411347517730496 0.2966101694915254


In [37]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.tokenized, df.toxic, test_size = 0.3, stratify = df.toxic)

In [38]:
padded_train = pad_sequences(x_train,maxlen = 40, padding = 'post', truncating = 'post')
padded_train= padded_train.reshape(3500,40)
padded_test = pad_sequences(x_test,maxlen = 40, truncating = 'post')
padded_test = padded_test.reshape(1500,40)

In [39]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train1 = sc.fit_transform(padded_train)
x_test1 = sc.transform(padded_test)

In [40]:
clas=SVC(kernel='linear', class_weight={1: 9}, C=30.0)
cl.fit(x_train1, y_train)
y_pred=cl.predict(x_test1)
cm1=confusion_matrix(y_test, y_pred)
cm1

array([[1297,   72],
       [ 126,    5]], dtype=int64)

In [41]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
acc=accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred)
rec=recall_score(y_test,y_pred)
print(acc,f1,rec)

0.868 0.04807692307692307 0.03816793893129771


In [58]:

def predans(comment):
    com=preprocess(comment)
    com1=' '.join(com) 
    l=[]
    l.append(com1)
    tokcom= tokenizer.texts_to_sequences(l)
    padcom=pad_sequences(tokcom,maxlen = 40, padding = 'post', truncating = 'post')
    ans=cl.predict(padcom)
    if ans>=0.5:
        return "toxic"
    else:
        return "not toxic"

In [59]:
comment=str(input())
value=predans(comment)
value

this movie displays how bad the current situation is in our country


'toxic'